In [8]:
import torch
import torchaudio
import polars as pl
import matplotlib.pyplot as plt
import os
import numpy as np

import helpers.input_processor as ip

In [2]:
#load data from dataframe
data_dir = "data/raw_training/training_data/"
target_label = 'murmur_in_recording'
df = (
    ip.loadTrainingData(data_dir)
    .filter(pl.col(target_label) != 'Unknown')
    .pipe(ip.encodeData)
    .select([
        pl.col('audio_file').apply(lambda x: os.path.join(data_dir, x)),
        pl.col(target_label)
    ])
)

loading data from save file:  cache/ingested_data.json


In [3]:
#balance the data so that there is an equal number of murmur positive and murmur negative samples
#do this by duplicating random rows of whichever group (pos or neg) is smaller
neg_df = df.filter(pl.col(target_label)==0.0)
pos_df = df.filter(pl.col(target_label)==1.0)
numNeg = neg_df.height
numPos = pos_df.height

while numNeg != numPos:
    if numNeg < numPos:
        df.vstack(neg_df.sample(n=min(numPos-numNeg, neg_df.height), shuffle=True), in_place=True)
    else: 
        df.vstack(pos_df.sample(n=min(numNeg-numPos, pos_df.height), shuffle=True), in_place=True)
    numNeg = df.filter(pl.col(target_label)==0.0).height
    numPos = df.filter(pl.col(target_label)==1.0).height

#reshuffle rows
df = df.sample(frac=1.0, shuffle=True)

#check number of positive and negative samples
numNeg = df.filter(pl.col(target_label)==0.0).height
numPos = df.filter(pl.col(target_label)==1.0).height
print('Total Samples:       ', df.height)
print('Positive Samples:    ', numPos)
print('Negative Samples:    ', numNeg)
print('Percent Positive Samples:    ', numPos/(numPos+numNeg))

Total Samples:        5328
Positive Samples:     2664
Negative Samples:     2664
Percent Positive Samples:     0.5


In [47]:
import librosa
import math, random
import numpy as np

# x = path to audio file
# duration = length of time (in seconds) to which the signal is resized
# sr = sample rate of the signal
# **kwargs = keyword args to pass to librosa.feature.melspectrogram()
def preprocessAudio(x, duration, channel, sr=4000, **kwargs):
    #read and load audio file in .wav format
    sig, samp_rate = librosa.load(x, sr=sr) #make sure that the correct sample rate is passed as a parameters. if unspecified, the function chooses some default value
    
    #resize sample, either by padding it with silence or truncating it
    sig = librosa.util.fix_length(sig, size=sr*duration)

    #implement audio augmentation:  ------------
    #time shift signal to the left or right by a random percent of its original length (max 99%)
    sig_len = sig.shape[0]
    max_shift = 0.99
    sig = np.roll(sig, round(random.random() * max_shift * sig_len))

    #convert to stereo
    sig = np.tile(sig, (2,1))

    #-------------------------------------------

    #get Mel spectrogram
    melSpec = librosa.feature.melspectrogram(y=sig, sr=samp_rate)
    melSpec = librosa.amplitude_to_db(melSpec)

    #implement image augmentation   ------------
    #-------------------------------------------

    return melSpec


#convert mono to stereo
def rechannel(sigIn, new_channel):
    sig = sigIn

    if (sig.shape[0] == new_channel):
      # Nothing to do
      return aud

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig))

In [74]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class AudioDataset(Dataset):
    def __init__(self, audioPaths, labels):
        self.audioPaths = audioPaths
        self.labels = labels
        self.sample_duration = 25
        self.n_mels = 128
        self.channel = 2

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        spec = torch.from_numpy(preprocessAudio(self.audioPaths[idx], self.sample_duration, self.channel, n_mels=self.n_mels))
        label = self.labels[idx]
        return spec, label

In [6]:
def splitDataset(ds, split_ratio=0.8):
    total_size = len(ds)
    train_size = round(split_ratio * total_size)
    test_size = total_size - train_size

    trainSet, testSet = torch.utils.data.random_split(ds, [train_size, test_size], generator=torch.Generator().manual_seed(0))
    return trainSet, testSet

In [75]:

audioPaths = df.get_column('audio_file').to_list()
labels = df.get_column(target_label).to_list()

ds = AudioDataset(audioPaths, labels)
train_data, test_data = splitDataset(ds)

# train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
# val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

_________________________________________________________________________________________________________
tutorial code
_________________________________________________________________________________________________________

In [76]:
batch_size = 64

train_ds = train_data
val_ds = test_data

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [77]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=10)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cuda', index=0)

In [78]:
# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].type(torch.LongTensor).to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        #if i % 10 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
    
    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=2   # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)

Epoch: 0, Loss: 2.12, Accuracy: 0.47
Epoch: 1, Loss: 1.63, Accuracy: 0.66
Finished Training


_________________________________________________________________________________________________________
Andre Code
_________________________________________________________________________________________________________

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    ## make a convolutional neural network
    def __init__(self):
        super(Net, self).__init__()
        self.sequence = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(1030400,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,10))
 
    def forward(self, x):
        return self.sequence(x)

net = Net()
net.cuda()

Net(
  (sequence): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=1030400, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_features=102

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [16]:
import tqdm

trainloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, prefetch_factor=4)
testloader = DataLoader(test_data, batch_size=64, shuffle=True)

for epoch in range(15):
    train_loss = 0.0
    epoch_bar = tqdm.tqdm(trainloader)
    epoch_bar.set_description("Epoch %s" % epoch)
    for i, data in enumerate(epoch_bar):
        epoch_bar.set_postfix({"loss": train_loss})
        
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass through model
        outputs = net(inputs)
        # Find the loss
        loss = criterion(outputs, labels)
        # Calculate the gradients
        loss.backward()
        # Update the weights
        optimizer.step()

        train_loss += loss.item()
    
        


Epoch 0:   0%|          | 0/134 [00:02<?, ?it/s, loss=0]


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 32, 128, 8] to have 1 channels, but got 32 channels instead